# Importing Libs

In [52]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

# Loading Resnet50 Model

In [53]:
model = tf.keras.applications.resnet50.ResNet50(weights='imagenet', include_top=False)

In [54]:
len(model.layers)

175

# Freezing major of first layers

In [55]:
freeze_to = 160
for layer in model.layers[:freeze_to]:
  layer.trainable = False


# Building the model

In [56]:
my_model = tf.keras.models.Sequential()

my_model.add(model)

my_model.add(tf.keras.layers.GlobalMaxPooling2D())

my_model.add(tf.keras.layers.Dense(units=256, activation='relu'))
my_model.add(tf.keras.layers.BatchNormalization())
my_model.add(tf.keras.layers.Dropout(0.3))

my_model.add(tf.keras.layers.Dense(units=256, activation='relu'))
my_model.add(tf.keras.layers.BatchNormalization())
my_model.add(tf.keras.layers.Dropout(0.3))

my_model.add(tf.keras.layers.Dense(units=120, activation='softmax'))

my_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

my_model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, None, None, 2048)  23587712  
_________________________________________________________________
global_max_pooling2d_1 (Glob (None, 2048)              0         
_________________________________________________________________
dense_28 (Dense)             (None, 256)               524544    
_________________________________________________________________
batch_normalization_18 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_29 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_19 (Batc (None, 256)             

## Some callbacks

In [77]:
import datetime
csv_logger = tf.keras.callbacks.CSVLogger('training.csv', separator=',', append=False)
# In each epoch Save mae, mse for training and testing data

checkpoints = tf.keras.callbacks.ModelCheckpoint(filepath='dog breeds-best.hdf5', verbose=1, save_best_only=True, save_weights_only=False)
# so that not saving every thing related to model (dropout activations, optimizer, loss, regularizers, ..) automatically 

log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,  histogram_freq=1)
# related to tensorboard

early_stopping = tf.keras.callbacks.EarlyStopping(monitor= 'loss' , min_delta=0.001, patience=2, mode = 'min')
# create early stopping for regularization

# Loading Dog Breeds dataset

In [58]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [59]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/gdrive/My Drive/Kaggle'

In [60]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [61]:
pwd

'/content/gdrive/My Drive/Kaggle'

In [62]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset

  0% 0.00/750M [00:00<?, ?B/s]
User cancelled operation


In [63]:
ls

 annotations/            images/       logs/                       training.csv
'dog breeds-best.hdf5'   kaggle.json   stanford-dogs-dataset.zip


In [64]:
!unzip \*.zip  && rm *.zip

Archive:  stanford-dogs-dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of *.zip or
        *.zip.zip, and cannot find stanford-dogs-dataset.zip.ZIP, period.

No zipfiles found.


In [65]:
from keras.preprocessing.image import ImageDataGenerator

dataset_path = '/content/gdrive/My Drive/Kaggle/images/Images'

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2) # set validation split

train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=32,
    subset='training', class_mode='categorical') # set as training data

validation_generator = train_datagen.flow_from_directory(
    dataset_path, # same directory as training data
    target_size=(224, 224),
    batch_size=32,
    subset='validation', class_mode='categorical')

Found 16508 images belonging to 120 classes.
Found 4072 images belonging to 120 classes.


In [66]:
train_generator.samples // 32

515

# Training the model

In [68]:
my_model.fit_generator(train_generator, steps_per_epoch = train_generator.samples // 32, 
                       validation_data = validation_generator, validation_steps = validation_generator.samples // 32,
                       epochs = 5, callbacks = [csv_logger, checkpoints, tensorboard_callback, early_stopping])

Epoch 1/5
 31/515 [>.............................] - ETA: 1:25 - loss: 5.3908 - accuracy: 0.0131

KeyboardInterrupt: ignored

In [69]:
## Accuracy is very bad (0.068 %)

#####################################################################################################

# let's try another pretrained model

In [71]:
model = tf.keras.applications.nasnet.NASNetLarge(weights='imagenet', include_top=False)

343613440/343610240 [==============================] - 4s 0us/step


In [72]:
len(model.layers)

1039

In [74]:
freeze_to = 1039
for layer in model.layers[:freeze_to]:
  layer.trainable = False


In [75]:
my_model = tf.keras.models.Sequential()

my_model.add(model)

my_model.add(tf.keras.layers.GlobalMaxPooling2D())

my_model.add(tf.keras.layers.Dense(units=256, activation='relu'))
my_model.add(tf.keras.layers.BatchNormalization())
my_model.add(tf.keras.layers.Dropout(0.3))

my_model.add(tf.keras.layers.Dense(units=256, activation='relu'))
my_model.add(tf.keras.layers.BatchNormalization())
my_model.add(tf.keras.layers.Dropout(0.3))

my_model.add(tf.keras.layers.Dense(units=120, activation='softmax'))

my_model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

my_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
NASNet (Model)               (None, 11, 11, 4032)      84916818  
_________________________________________________________________
global_max_pooling2d_3 (Glob (None, 4032)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 256)               1032448   
_________________________________________________________________
batch_normalization_22 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_22 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_23 (Batc (None, 256)             

In [79]:
my_model.fit_generator(train_generator, steps_per_epoch = train_generator.samples // 32, 
                       validation_data = validation_generator, validation_steps = validation_generator.samples // 32,
                       epochs = 1, callbacks = [csv_logger, checkpoints, tensorboard_callback, early_stopping])

515/515 [==============================] - ETA: 0s - loss: 0.3375 - accuracy: 0.8936
Epoch 00001: val_loss did not improve from 0.58875
515/515 [==============================] - 163s 317ms/step - loss: 0.3375 - accuracy: 0.8936 - val_loss: 0.6421 - val_accuracy: 0.8524


In [80]:
import pandas as pd
pd.read_csv('training.csv')

,Unnamed: 0,0.8936027884483337,0.33754846453666687,0.8523622155189514,0.6420602798461914


In [81]:
from sklearn.metrics import confusion_matrix , classification_report
y_pred = my_model.predict_classes(validation_generator)
print(classification_report(y_test, y_pred))
pd.DataFrame(confusion_matrix(y_test, y_pred))#, columns=['negative', 'positive'], index = ['negative', 'positive'])

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


NameError: ignored